# Import

In [1]:
import numpy as np,pandas as pd
import matplotlib.pyplot as mpl,seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
import numpy as np,pandas as pd
from scipy.linalg import block_diag
from sklearn.utils import check_random_state
import yfinance as yf

# Import py file

In [2]:
from Clustering import*

# Data

In [3]:
allStocks = pd.DataFrame()
stocks = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA", "FB", "JPM", "UNH",
"V", "JNJ", "HD", "WMT", "PG", "BAC", "MA", "PFE", "DIS", "AVGO",
"XOM", "ACM", "CSCO", "NFLX", "NKE", "LLY", "KO", "TMO", "CRM", "COST", "AAL", "TWTR"]
for symb in stocks:
    data = pd.DataFrame(yf.Ticker(symb).history(start = "2019-01-01", end = "2021-08-08")['Close']).transpose()
    # data.save_batch_run(symb + '.csv')
    if symb == 'BTC-USD':
      allStocks = data.copy()
    else:
      allStocks=allStocks.append(data)
allStocks.index = [stocks]
allStocks = allStocks.transpose()
allStocks = allStocks.pct_change(1)

In [4]:
allStocks.head(10)

,AAPL,MSFT,GOOG,AMZN,TSLA,FB,JPM,UNH,V,JNJ,...,CSCO,NFLX,NKE,LLY,KO,TMO,CRM,COST,AAL,TWTR
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,0.001141,-0.004431,0.009888,0.024741,-0.068149,0.035014,0.017312,-0.022599,0.007428,-0.010074,...,-0.008770,0.000000,-0.001079,-0.007259,-0.008870,-0.019527,-0.010367,0.005154,0.011523,0.002436
2019-01-03,-0.099607,-0.036788,-0.028484,-0.025241,-0.031472,-0.029039,-0.014212,-0.027270,-0.036037,-0.015890,...,-0.036368,0.013226,-0.017688,-0.031076,-0.006179,-0.033042,-0.037993,-0.021195,-0.074507,-0.028462
2019-01-04,0.042689,0.046509,0.053786,0.050064,0.057697,0.047138,0.036865,0.011695,0.043081,0.016783,...,0.045045,0.097234,0.026117,0.030096,0.019940,0.045200,0.057976,0.029039,0.065868,0.070025
2019-01-07,-0.002226,0.001276,-0.002167,0.034353,0.054361,0.000725,0.000695,0.001920,0.018032,-0.006415,...,0.006756,0.059717,0.014334,0.005407,-0.013033,0.013483,0.030878,0.003685,0.028402,0.046411
2019-01-08,0.019063,0.007251,0.007385,0.016612,0.001164,0.032452,-0.001886,0.013371,0.005439,0.023226,...,0.008100,0.015634,0.013339,0.009195,0.011288,0.023270,0.024610,0.007488,-0.016085,0.014678
2019-01-09,0.016982,0.014299,-0.001505,0.001714,0.009483,0.011927,-0.001690,0.001439,0.011769,-0.007926,...,-0.005510,-0.000968,-0.001825,0.007478,-0.019166,0.027872,0.001853,0.006473,0.030845,0.014151
2019-01-10,0.003196,-0.006426,-0.004029,-0.001928,0.019023,-0.000208,-0.000100,0.005048,0.001878,0.006050,...,-0.001847,0.014689,-0.002220,-0.010835,0.010736,0.010449,0.007192,0.003526,-0.041293,0.026047
2019-01-11,-0.009818,-0.007722,-0.012277,-0.009455,0.006638,-0.002774,-0.004781,0.011435,-0.004399,0.000308,...,0.005782,0.039826,-0.004973,0.002070,0.005736,0.004773,0.003469,-0.000617,-0.007491,-0.006649


# Snippet 4-1

In [5]:
correlation = allStocks.corr()
correlation

,AAPL,MSFT,GOOG,AMZN,TSLA,FB,JPM,UNH,V,JNJ,...,CSCO,NFLX,NKE,LLY,KO,TMO,CRM,COST,AAL,TWTR
AAPL,1.000000,0.783626,0.677517,0.659740,0.448578,0.672470,0.460121,0.499210,0.635290,0.485567,...,0.605548,0.495635,0.520225,0.383504,0.451567,0.556896,0.572799,0.587520,0.244686,0.484427
MSFT,0.783626,1.000000,0.777213,0.712672,0.455468,0.685031,0.510405,0.578594,0.735244,0.562657,...,0.660020,0.545813,0.556056,0.458551,0.505662,0.607901,0.696877,0.646853,0.211789,0.500800
GOOG,0.677517,0.777213,1.000000,0.644693,0.384036,0.717569,0.501285,0.527876,0.682056,0.473382,...,0.568042,0.503144,0.509968,0.374864,0.481173,0.496845,0.594321,0.524321,0.250200,0.517763
AMZN,0.659740,0.712672,0.644693,1.000000,0.414600,0.646083,0.240575,0.352817,0.441610,0.330359,...,0.452241,0.612933,0.359710,0.289699,0.264070,0.450726,0.561712,0.529454,0.109002,0.445669
TSLA,0.448578,0.455468,0.384036,0.414600,1.000000,0.351005,0.253864,0.230596,0.372638,0.149913,...,0.262250,0.362388,0.290441,0.092833,0.205354,0.270043,0.434153,0.279371,0.208334,0.306701
FB,0.672470,0.685031,0.717569,0.646083,0.351005,1.000000,0.382889,0.452138,0.565329,0.382042,...,0.483329,0.511512,0.436250,0.304107,0.340125,0.450604,0.591228,0.475088,0.173143,0.547961
JPM,0.460121,0.510405,0.501285,0.240575,0.253864,0.382889,1.000000,0.535839,0.680552,0.475779,...,0.556237,0.160963,0.568749,0.282689,0.604456,0.322162,0.376574,0.346699,0.554918,0.370885
UNH,0.499210,0.578594,0.527876,0.352817,0.230596,0.452138,0.535839,1.000000,0.605010,0.564607,...,0.504888,0.289053,0.517759,0.460799,0.490799,0.453913,0.397483,0.477981,0.257782,0.329831
V,0.635290,0.735244,0.682056,0.441610,0.372638,0.565329,0.680552,0.605010,1.000000,0.564701,...,0.644609,0.330238,0.643634,0.385290,0.646363,0.528642,0.582648,0.511558,0.391912,0.483516
JNJ,0.485567,0.562657,0.473382,0.330359,0.149913,0.382042,0.475779,0.564607,0.564701,1.000000,...,0.593599,0.250373,0.458486,0.540450,0.583800,0.537719,0.371512,0.533507,0.196385,0.284560


In [6]:
correlationNew, clusters, silh = clusterKMeansBase(correlation)
clusters

{0: [('AAPL',),
  ('MSFT',),
  ('GOOG',),
  ('JPM',),
  ('UNH',),
  ('V',),
  ('JNJ',),
  ('HD',),
  ('WMT',),
  ('PG',),
  ('BAC',),
  ('MA',),
  ('PFE',),
  ('DIS',),
  ('AVGO',),
  ('XOM',),
  ('ACM',),
  ('CSCO',),
  ('NKE',),
  ('LLY',),
  ('KO',),
  ('TMO',),
  ('COST',)],
 1: [('AMZN',), ('TSLA',), ('FB',), ('NFLX',), ('CRM',), ('AAL',), ('TWTR',)]}

# Snippet 4-2

In [7]:
corr, clusters, silh = clusterKMeansTop(correlation, numberClusters = None, iterations = 10)
clusters

{0: [('AAPL',),
  ('MSFT',),
  ('GOOG',),
  ('AMZN',),
  ('TSLA',),
  ('FB',),
  ('AVGO',),
  ('NFLX',),
  ('CRM',),
  ('TWTR',)],
 1: [('UNH',),
  ('HD',),
  ('CSCO',),
  ('JPM',),
  ('V',),
  ('BAC',),
  ('MA',),
  ('DIS',),
  ('XOM',),
  ('ACM',),
  ('NKE',),
  ('KO',),
  ('AAL',)],
 2: [('JNJ',), ('WMT',), ('PG',), ('PFE',), ('LLY',), ('TMO',), ('COST',)]}

# Snippet 4-3

In [8]:
correlation = randomBlockCorrelation(100, 4)
sns.heatmap(correlation, cmap = "viridis")

<AxesSubplot:>